In [ ]:
import pandas as pd
import gensim
from gensim import corpora
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
from pprint import pprint
%matplotlib inline

In [ ]:
df = pd.read_pickle('./39570/vectorizers/Bert_LRG___Chobs_context_5_gap_length_20.pkl')
df_normal = pd.read_pickle('./39570/clusterers/Bert_LRG___Chobs_context_5_gap_length_20/tsne/DBscan_min_samples_4_eps_1.5.pkl')

In [ ]:
df_cut = df[['left_token_str_clean', 'ins_tokens_str_clean', 'right_token_str_clean']]
corpus = []
for i, row in df_cut.iterrows():
    one_sent = row[0] + ' ' + row[1] + ' ' + row[2]
    corpus.append([i for i in one_sent.split() if i not in stop])
print(corpus)

In [ ]:
#all tokens (except for stopwords)
dictionary = corpora.Dictionary(corpus)
doc_term_matrix = [dictionary.doc2bow(sent) for sent in corpus]
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=100, id2word = dictionary, passes=50)
pprint(ldamodel.print_topics(num_topics=10, num_words=10))

In [ ]:
counter = 0 #total number of tokens
for sent in corpus:
    counter += len(sent)
counter

In [ ]:
#deleting those tokens that appear only once
all_tokens = sum(corpus, [])
tokens_once = set(word for word in set(all_tokens) if all_tokens.count(word) == 1)
sents = [[word for word in text if word not in tokens_once]
         for text in corpus]

In [ ]:
counter = 0 #number of tokens that appear more than once
for sent in sents:
    counter += len(sent)
counter

In [ ]:
dictionary = corpora.Dictionary(sents)
doc_term_matrix = [dictionary.doc2bow(sent) for sent in sents]
Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda(doc_term_matrix, num_topics=100, id2word = dictionary, passes=50)
pprint(ldamodel.print_topics(num_topics=10, num_words=10))

In [ ]:
dictionary = corpora.Dictionary(sents)
corpus = [dictionary.doc2bow(text) for text in sents]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=80, 
                                           #random_state=100,
                                           #update_every=1,
                                           chunksize=100,
                                           passes=10)

In [ ]:
pprint(lda_model.print_topics(num_topics=10, num_words=10))
doc_lda = lda_model[corpus]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=sents, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, mds='tsne')
vis

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start, step): #DOES NOT WORK, ISSUE WITH GENSIM
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=10, chunksize=100)
        print(model)
        model_list.append(model)
        coherence_model_lda = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
        print(coherence_lda)
        coherence_values.append(coherence_lda)

    return model_list, coherence_values

In [ ]:
#model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=sents, start=60, limit=100, step=5)
#coherence_values = []
model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=100, passes=10, chunksize=100) #MANUALLY CHANGE NUM_TOPICS
coherence_model_lda = CoherenceModel(model=model, texts=sents, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
coherence_values.append(coherence_lda)

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=sents, start=20, limit=100, step=10)

In [ ]:
start=20; limit=100; step=10
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=sents, start=30, limit=40, step=1)

In [ ]:
lda_model_updated = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=37, 
                                           #random_state=100,
                                           #update_every=1,
                                           chunksize=100,
                                           passes=10)
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_updated, corpus, dictionary, mds='tsne')
vis

In [ ]:
import ipywidgets as widgets
from IPython.display import display
lda_token = widgets.Text(
    value=None,
    description = 'Token:',
    continuous_update = False
)

def w_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print('For ', lda_token.value, ': ', lda_model_updated.get_term_topics(dictionary.doc2idx([lda_token.value]), minimum_probability=None))


lda_token.observe(w_on_change)
display(lda_token)

In [ ]:
print(lda_model_updated.print_topic(4))

In [ ]:
doc2bow_func = lambda x: dictionary.doc2bow(x) 
doc_term_matrix = list(map(doc2bow_func, sents))

In [ ]:
get_topics = lambda x: lda_model_updated.get_document_topics(x, minimum_probability=0)
result = list(map(get_topics, doc_term_matrix))

In [ ]:
topic_to_chobj = []
for chobj in result:
    topic_to_chobj.append(max(chobj,key=lambda item:item[1]))

In [ ]:
def get_token_ids(sents):
    token_to_ids = []
    for obj in sents:
        id_unique = dictionary.doc2idx(obj)
        token_to_ids.append(id_unique)
    return token_to_ids

def get_topics(token_to_ids):
    topics_all_chobj = []
    one_topic_dict = []
    for chobj in token_to_ids:
        topic_dict = {}
        
        counter_max = 0
        for token_id in chobj:
            topic_pr = lda_model_updated.get_term_topics(token_id, minimum_probability=None)
            if len(topic_pr) == 1 and len(topic_pr[0])>0: #when a word has only one topic assigned to it and it is not null
                if str(topic_pr[0][0]) not in topic_dict.keys(): # if a topic is encountered for the first time
                    topic_dict[str(topic_pr[0][0])] = (topic_pr[0][1], 1)
                    if counter_max == 0:
                        counter_max = 1
                        topic_max = topic_pr[0][0]
                        prob_max = topic_pr[0][1]
                elif str(topic_pr[0][0]) in topic_dict.keys(): #if a topic has already been encountered 
                    pr_counter = (topic_dict[str(topic_pr[0][0])][0] + topic_pr[0][1], topic_dict[str(topic_pr[0][0])][1] + 1)
                    topic_dict[str(topic_pr[0][0])] = pr_counter
                    if pr_counter[1] > counter_max :
                        counter_max = pr_counter[1]
                        topic_max = str(topic_pr[0][0])
                        prob_max = pr_counter[0]
                    elif pr_counter[1] == counter_max :
                        if pr_counter[0] > prob_max:
                            prob_max = pr_counter[0]
                            topic_max = str(topic_pr[0][0])
            elif len(topic_pr) > 1: #when a word has more than one topic assigned to it
                pr_max = topic_pr[0][1]
                for i in range(1,len(topic_pr)): #to choose the topic with max probability, MAYBE NEEDS OTHER METHOD LIKE ADDING ALL TOPICS
                    if topic_pr[i][1] > pr_max:
                        tuple_max = topic_pr[i]
                if str(tuple_max[0]) not in topic_dict.keys():
                    topic_dict[str(tuple_max[0])] = (tuple_max[1], 1)
                    counter_max = 1
                    topic_max = str(tuple_max[0])
                    prob_max = tuple_max[1]
                elif str(tuple_max[0]) in topic_dict.keys():
                    pr_counter = (topic_dict[str(tuple_max[0])][0] + tuple_max[1], topic_dict[str(tuple_max[0])][1] + 1)
                    if pr_counter[1] > counter_max :
                        counter_max = pr_counter[1]
                        topic_max = str(tuple_max[0])
                        prob_max = pr_counter[0]
                    elif pr_counter[1] == counter_max :
                        if pr_counter[0] > prob_max:
                            prob_max = pr_counter[0]
                            topic_max = str(tuple_max[0])
                    topic_dict[str(tuple_max[0])] = pr_counter

        one_topic_dict.append((topic_max, prob_max))
        topics_all_chobj.append(topic_dict)
    return topics_all_chobj, one_topic_dict

token_to_ids = get_token_ids(sents)
topics_all_chobj, one_topic_by_occur = get_topics(token_to_ids)

In [ ]:
def get_norm_probabilities(topics_all_chobj):
# a method for normalizing probabilities (*number of occurences of a topic in chobj / total number of occurences of all topics in chobj)
    norm_topics_all_chobj = [] 
    for chobj in topics_all_chobj:
        chobj_counter = 0
        for k, v in chobj.items():
            chobj_counter += v[1]
        for k in chobj.keys():
            norm_prob = chobj[k][0] * chobj[k][1] / chobj_counter 
            chobj[k] = norm_prob
        norm_topics_all_chobj.append(chobj)
    return norm_topics_all_chobj

def get_one_topic_by_prob(topics_all_chobj):
# a method for choosing a topic with the highest probability
    one_topic_all_chobs = []
    for chobj in norm_topics_all_chobj:
        if len(chobj) > 1:
            topic_max = max(chobj, key=chobj.get)
            pair_max = (topic_max, chobj[topic_max])
        elif len(chobj) == 1:
            pair_max = next(iter( chobj.items() ))
        one_topic_all_chobs.append(pair_max)
    
    return one_topic_all_chobs

norm_topics_all_chobj = get_norm_probabilities(topics_all_chobj)
one_topic_by_prob = get_one_topic_by_prob(norm_topics_all_chobj)

In [ ]:
df['clusters'] = [int(i[0]) for i in topic_to_chobj]
df['t-SNE-X'] = df_normal['t-SNE-X']
df['t-SNE-Y'] = df_normal['t-SNE-Y']
max(df['clusters'])

In [ ]:
import matplotlib.pyplot as plt
data = df['clusters'].value_counts()
data.plot.bar()


In [ ]:
import plotly.io
import plotly
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import random
init_notebook_mode(connected=True)

r = lambda: random.randint(0,255)
plot_data = df.copy()
if 't-SNE-X' not in plot_data.columns:
    print('TSNE coordinates have not been added. Please run the desired clusterizer again in the corresponding notebook')
else:
    traces = []
    for c in plot_data.clusters.unique():
        trace = go.Scatter(
            x=plot_data[plot_data["clusters"]==c]["t-SNE-X"],
            y=plot_data[plot_data["clusters"]==c]["t-SNE-Y"],
            mode = 'markers',
            name = str(c),
            marker = go.scatter.Marker(size=4, color='#%02X%02X%02X' % (r(),r(),r())),
            showlegend = True,
        )
        traces.append(trace)

    data = traces

    # Plot and embed in ipython notebook
    #fname = '../' + d.value + '/figures/' + vec + clu[:-4] 

    plotly.offline.iplot(data,image_width=1280, image_height=800, image='png', filename='plot_image')

    #plotly.offline.plot(data, filename=fname+'.html', auto_open=False,
                        #image_width=1280, image_height=800)
                        #,image='png', image_filename='plot_image')

In [ ]:
df.Bulk.value_counts()

In [ ]:
df.loc[df['birth_place'].notnull(),"clusters"].value_counts()

In [ ]:
df.loc[(df['birth_place'] == "Y") & (df['Bulk'] == "N"),"clusters"].value_counts()

In [ ]:
df.loc[(df['birth_place'] == "N") & (df['Bulk'] == "N"),"clusters"].value_counts()